# Script for cleansing / manipulating raw pakkeleg data

- The steps in this script are: 1. create action/victim vector

In [282]:
import pandas as pd
import numpy as np


In [456]:
# import data, ; as the seperator
raw_annotaitons = pd.read_csv('/Users/annadaugaard/Documents/1.sem/decision_making/DM23/pakkeleg_annotations.csv', sep = ';')
#split data into two rounds
round_1 = raw_annotaitons[raw_annotaitons['Round'] == 1] 
round_2 = raw_annotaitons[raw_annotaitons['Round'] == 2] 

In [457]:
actions_ = round_1.drop(['Round'], axis=1)

# Create an empty list 
action_vectors =[] 
  
# Iterate over each row and append the values from each player to a vector and append vector to list
for index, rows in actions_.iterrows(): 
     my_list =[rows.Niels, rows.Daniel, rows.Mina, rows.Emma, rows.Anton, rows.Jess, rows.Aleks, rows.Sille]
     action_vectors.append(my_list) 
     
# lag the vector by 1 to create vectors on the format : [0,0,0,0,1,-1,0] bla bla bla... for easily extract the player and the victim
action_vectors_lagged = [[0,0,0,0,0,0,0]] + action_vectors

actions = [list(map(lambda x, y: x - y, ii, jj)) for ii, jj in zip(action_vectors,action_vectors_lagged)]

# making action, victim vector pairs alongisde with the time index:
action_pairs = []
fucked_up = []
for idx, i in enumerate(actions):
    try:
        victim = i.index(-1) +1 # +1 is due to nulindeksering... it is hard to interpret if it is nulindekseret
        actor = i.index(1) +1
        pair = [actor,victim,idx-1]
        action_pairs.append(pair)
        
    except:
        fucked_up.append([i,idx]) # to catch annotation errors (e.g. if there is only a 1 in the vector and not a corresponding -1)

df = pd.DataFrame(action_pairs, columns = ['player', 'victim','t']) #add the columns 
merged_df_round1 = pd.merge(round_1, df, on='t')
merged_df_round1

# note: tanke om action skal ligge "ved siden af status", eller om vi skal have dem som +/-1... her bliver de lagt "ved siden af"

,Niels,Daniel,Mina,Emma,Anton,Jess,Aleks,Sille,Round,t,player,victim
0,2,0,1,2,3,1,1,2,1,11,5,4
1,2,0,1,1,4,1,1,2,1,12,6,5
2,2,0,1,1,3,2,1,2,1,13,3,4
3,2,0,2,0,3,2,1,2,1,14,4,5
4,2,0,2,1,2,2,1,2,1,15,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...
110,2,2,4,0,0,1,1,2,1,122,7,3
111,2,2,3,0,0,1,2,2,1,123,7,3
112,2,2,2,0,0,1,3,2,1,124,6,3
113,2,2,1,0,0,2,3,2,1,125,1,2


In [458]:
#distribute rounds pr. participant (remember, nulindeksering... players + victims er 1-indekseret, så man skal trække 1 fra for at få 0-indeks)

rounds_pr_participant = [list() for _ in range(8)]
for index, rows in merged_df_round1.iterrows(): 
    my_list =[[rows.Niels, rows.Daniel, rows.Mina, rows.Emma, rows.Anton, rows.Jess, rows.Aleks, rows.Sille],rows.victim,rows.t]
    rounds_pr_participant[rows.player-1].append(my_list) #create 8 lists with each participants actions in each (her er nulindekseret)

In [468]:
#distribute hate (remember, nulindeksering)
hate_vectors = [list() for _ in range(8)]
for index, rows in merged_df_round1.iterrows(): 
    i = [0 for element in range(8)]
    i[rows.player-1] = 1 #we subtract 1 to "go back" to nulindeksering again
    hate_vectors[rows.victim-1].append([i,rows.t])

In [527]:
#function to merge hate vectors with players turns

def merge_vectors(rounds_pr_participant, hate_vectors):
    # Initialize an empty list to store the final result
    merged_vectors = []

    # For each round, find the corresponding hate vector
    for round_info in rounds_pr_participant:
        # Extract the time from the round info
        round_time = round_info[2]

        # Initialize a default hate vector
        hate_vector = [[0, 0, 0, 0, 0, 0, 0, 0], round_time]

        # Search for a matching hate vector in reversed order (find the latest hatevector based on the t of the players turn)
        for hv in reversed(hate_vectors):
            if hv[1] <= round_time:
                hate_vector = hv
                break

        # Append the round info and hate vector to the final result
        merged_vectors.append([round_info, hate_vector])

    return merged_vectors


result = merge_vectors(rounds_pr_participant[1], hate_vectors[1])

result

[[[[2, 0, 1, 0, 2, 2, 2, 3], 6, 18], [[0, 0, 0, 0, 0, 0, 0, 0], 18]],
 [[[1, 1, 2, 0, 2, 1, 2, 3], 3, 21], [[0, 0, 0, 0, 0, 0, 0, 0], 21]],
 [[[2, 2, 1, 0, 3, 2, 2, 0], 5, 26], [[0, 0, 0, 0, 0, 0, 0, 0], 26]],
 [[[2, 2, 2, 2, 0, 2, 2, 0], 4, 31], [[1, 0, 0, 0, 0, 0, 0, 0], 30]],
 [[[0, 2, 2, 3, 1, 0, 2, 2], 4, 43], [[0, 0, 0, 1, 0, 0, 0, 0], 36]],
 [[[1, 1, 1, 1, 1, 1, 2, 4], 8, 56], [[0, 0, 0, 0, 1, 0, 0, 0], 48]],
 [[[1, 2, 1, 1, 1, 1, 3, 2], 4, 60], [[0, 0, 0, 0, 1, 0, 0, 0], 48]],
 [[[1, 3, 1, 0, 2, 1, 1, 3], 8, 63], [[0, 0, 0, 0, 1, 0, 0, 0], 48]],
 [[[1, 3, 1, 1, 2, 1, 1, 2], 4, 65], [[0, 0, 0, 1, 0, 0, 0, 0], 64]],
 [[[1, 1, 3, 0, 3, 2, 1, 1], 3, 75], [[0, 0, 0, 0, 0, 1, 0, 0], 72]],
 [[[1, 1, 2, 1, 2, 1, 3, 1], 7, 85], [[0, 0, 0, 1, 0, 0, 0, 0], 77]],
 [[[2, 2, 2, 1, 2, 1, 1, 1], 5, 88], [[0, 0, 0, 1, 0, 0, 0, 0], 77]],
 [[[1, 2, 1, 1, 2, 2, 1, 2], 6, 92], [[0, 0, 0, 0, 1, 0, 0, 0], 91]],
 [[[2, 1, 2, 0, 3, 2, 1, 1], 5, 99], [[0, 0, 0, 0, 0, 1, 0, 0], 97]],
 [[[2, 1, 1, 1, 1, 2

In [535]:
#extract the actions to copy to julia recovery script
actions = []
for i in result:
    actions.append(i[0][1])
actions

[6, 3, 5, 4, 4, 8, 4, 8, 4, 3, 7, 5, 6, 5, 8, 6, 3]

In [534]:
# have the data in the right format with the package status vector on left and the hate vector on right
data_for_model = []
for i in result:
    data_for_model.append([i[0][0],i[1][0]])
data_for_model

[[[2, 0, 1, 0, 2, 2, 2, 3], [0, 0, 0, 0, 0, 0, 0, 0]],
 [[1, 1, 2, 0, 2, 1, 2, 3], [0, 0, 0, 0, 0, 0, 0, 0]],
 [[2, 2, 1, 0, 3, 2, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0]],
 [[2, 2, 2, 2, 0, 2, 2, 0], [1, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 2, 2, 3, 1, 0, 2, 2], [0, 0, 0, 1, 0, 0, 0, 0]],
 [[1, 1, 1, 1, 1, 1, 2, 4], [0, 0, 0, 0, 1, 0, 0, 0]],
 [[1, 2, 1, 1, 1, 1, 3, 2], [0, 0, 0, 0, 1, 0, 0, 0]],
 [[1, 3, 1, 0, 2, 1, 1, 3], [0, 0, 0, 0, 1, 0, 0, 0]],
 [[1, 3, 1, 1, 2, 1, 1, 2], [0, 0, 0, 1, 0, 0, 0, 0]],
 [[1, 1, 3, 0, 3, 2, 1, 1], [0, 0, 0, 0, 0, 1, 0, 0]],
 [[1, 1, 2, 1, 2, 1, 3, 1], [0, 0, 0, 1, 0, 0, 0, 0]],
 [[2, 2, 2, 1, 2, 1, 1, 1], [0, 0, 0, 1, 0, 0, 0, 0]],
 [[1, 2, 1, 1, 2, 2, 1, 2], [0, 0, 0, 0, 1, 0, 0, 0]],
 [[2, 1, 2, 0, 3, 2, 1, 1], [0, 0, 0, 0, 0, 1, 0, 0]],
 [[2, 1, 1, 1, 1, 2, 1, 3], [0, 0, 0, 0, 1, 0, 0, 0]],
 [[2, 2, 1, 1, 1, 2, 1, 2], [0, 0, 0, 0, 1, 0, 0, 0]],
 [[2, 1, 5, 0, 0, 1, 1, 2], [0, 0, 1, 0, 0, 0, 0, 0]]]

In [361]:
#ROUND 2... basically the same 
actions_ = round_2.drop(['Round'], axis=1)
# Create an empty list 
action_vectors =[] 
  
# Iterate over each row 
for index, rows in actions_.iterrows(): 
     my_list =[rows.Niels, rows.Daniel, rows.Mina, rows.Emma, rows.Anton, rows.Jess, rows.Aleks, rows.Sille]
     action_vectors.append(my_list) 
     
# add "index change" to the list of vectors.
action_vectors_lagged = [[0,0,0,0,0,0,0]] + action_vectors

actions = [list(map(lambda x, y: x - y, ii, jj)) for ii, jj in zip(action_vectors,action_vectors_lagged)]

# making action, victim vector pairs alongisde with the time index:
action_pairs = []
fucked_up = []
for idx, i in enumerate(actions):
    try:
        victim = i.index(-1) +1
        actor = i.index(1) +1
        pair = [actor,victim,idx-1]
        action_pairs.append(pair)
        
    except:
        fucked_up.append([i,idx])
        #print(i, idx)
        
df = pd.DataFrame(action_pairs, columns = ['player', 'victim','t']) 
merged_df_round2 = pd.merge(round_2, df, on='t')
merged_df_round2

# note: tanke om action skal ligge "ved siden af status", eller om vi skal have dem som +/-1

,Niels,Daniel,Mina,Emma,Anton,Jess,Aleks,Sille,Round,t,player,victim
0,3,0,2,0,2,2,0,3,2,0,3,8
1,3,0,3,0,2,2,0,2,2,1,5,1
2,2,0,3,0,3,2,0,2,2,2,8,3
3,2,0,2,0,3,2,0,3,2,3,2,5
4,2,1,2,0,2,2,0,3,2,4,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...
138,2,0,2,1,1,2,2,2,2,138,7,3
139,2,0,1,1,1,2,3,2,2,139,2,1
140,1,1,1,1,1,2,3,2,2,140,4,7
141,1,1,1,2,1,2,2,2,2,141,3,7
